In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor  # Import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('train_data.csv')

# Replace all occurrences of -1 with NaN in the entire DataFrame as they are considered invalid
df.replace(-1, np.nan, inplace=True)


# Function to replace outliers with the median
def fill_outliers_with_median(df, numerical_columns):
    for col in numerical_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Replace outliers with median value of the column
        median_value = df[col].median()
        df[col] = df[col].apply(lambda x: median_value if x < lower_bound or x > upper_bound else x)
    return df

# Apply filling outliers with the median
numerical_columns = ['che_pc_usd', 'che_perc_gdp', 'insurance_perc_che', 'population',
                     'prev_perc', 'price_month', 'price_unit', 'public_perc_che', 'target']
df_imputed = fill_outliers_with_median(df_imputed, numerical_columns)

# Check the result after filling outliers with the median
print("\nData after filling outliers with median:")
print(df_imputed.head())

# Preprocessing pipeline
categorical_columns = ['brand', 'country', 'cluster_nl', 'indication', 'therapeutic_area']
numerical_columns = ['che_pc_usd', 'che_perc_gdp', 'population', 'price_month', 'price_unit',
                     'public_perc_che', 'prev_perc', 'price_month', 'price_unit', 'year', 'month',
                      'day', 'day_of_week', 'days_since_launch']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Impute missing numerical values with median
            ('scaler', StandardScaler())  # Scale numerical data
        ]), numerical_columns),

        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing categorical values with the most frequent value
            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))  # Handle unknown categories
        ]), categorical_columns)
    ])

# Convert date columns to datetime format
df_imputed['date'] = pd.to_datetime(df_imputed['date'], errors='coerce')
df_imputed['ind_launch_date'] = pd.to_datetime(df_imputed['ind_launch_date'], errors='coerce')

# Impute missing 'ind_launch_date' with mode
df_imputed['ind_launch_date'] = df_imputed['ind_launch_date'].fillna(df_imputed['ind_launch_date'].mode()[0])

# Extract date-related features
df_imputed['year'] = df_imputed['date'].dt.year
df_imputed['month'] = df_imputed['date'].dt.month
df_imputed['day'] = df_imputed['date'].dt.day
df_imputed['day_of_week'] = df_imputed['date'].dt.dayofweek
df_imputed['days_since_launch'] = (df_imputed['date'] - df_imputed['ind_launch_date']).dt.days

# Define features and target
X = df_imputed.drop(columns=['target', 'date', 'ind_launch_date'])  # Drop the target and date-related columns
y = df_imputed['target']  # Target variable (monthly sales)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model (Decision Tree Regressor)
model = DecisionTreeRegressor(random_state=66)  # Initialize DecisionTreeRegressor

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Handle outliers in the submission data
test_df = pd.read_csv('submission_data.csv')

# Apply the same transformations as done with the training data
test_df['date'] = pd.to_datetime(test_df['date'], errors='coerce')
test_df['ind_launch_date'] = pd.to_datetime(test_df['ind_launch_date'], errors='coerce')
test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day
test_df['day_of_week'] = test_df['date'].dt.dayofweek
test_df['days_since_launch'] = (test_df['date'] - test_df['ind_launch_date']).dt.days

# Apply filling outliers with the median on the test data
test_df = fill_outliers_with_median(test_df, numerical_columns)

# Apply the same preprocessing pipeline to the test data
test_predictions = pipeline.predict(test_df.drop(columns=['date', 'ind_launch_date']))

# Load the submission template
submission_template = pd.read_csv('submission_template.csv')

# Ensure prediction order matches the template
submission_template['prediction'] = test_predictions  # Fill in predictions

# Save the final submission file
submission_template.to_csv('final_submissionDT_.csv', index=False)
print("Submission saved to 'final_submissionDT_.csv'")



Data after filling outliers with median:
        brand  che_pc_usd  che_perc_gdp               cluster_nl corporation  \
0  BRAND_354E    1.209114      1.665879  BRAND_354E_COUNTRY_88A3   CORP_D524   
1  BRAND_626D         NaN           NaN  BRAND_626D_COUNTRY_8B47   CORP_01C7   
2  BRAND_45D9    1.209114      1.665879  BRAND_45D9_COUNTRY_88A3   CORP_39F7   
3  BRAND_D724    1.851280      2.051770  BRAND_D724_COUNTRY_445D   CORP_711A   
4  BRAND_4887    1.791199      2.059130  BRAND_4887_COUNTRY_D8B0   CORP_443D   

        country launch_date       date       drug_id ind_launch_date  ...  \
0  COUNTRY_88A3  2014-06-01 2014-06-01  DRUG_ID_8795      2019-11-01  ...   
1  COUNTRY_8B47  2014-06-01 2014-06-01  DRUG_ID_E66E      2014-09-01  ...   
2  COUNTRY_88A3  2014-06-01 2014-06-01  DRUG_ID_F272      2019-11-01  ...   
3  COUNTRY_445D  2014-06-01 2014-06-01  DRUG_ID_1D4E      2019-11-01  ...   
4  COUNTRY_D8B0  2014-06-01 2014-06-01  DRUG_ID_AA88      2019-11-01  ...   

  price_month 

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
